# 文本转语音 (Text-to-speech, TTS)

文本转语音（TTS）是将文字转换成自然语音的任务，这种语音可以支持多种语言和多个说话者。目前，🤗 Transformers 中已经提供了多种文本转语音模型，比如 [Bark](https://huggingface.co/docs/transformers/main/en/model_doc/bark)、[MMS](https://huggingface.co/docs/transformers/main/en/model_doc/mms)、[VITS](https://huggingface.co/docs/transformers/main/en/model_doc/vits) 和 [SpeechT5](https://huggingface.co/docs/transformers/main/en/model_doc/speecht5)。

你可以通过 `text-to-audio` pipeline（或者它的别名——`text-to-speech`）轻松生成音频。

有些模型，比如 Bark 还可以生成非语言语音，比如笑声、叹息和哭泣，甚至可以添加音乐。下面是一个使用 Bark 进行“文本转语音”的示例：

In [ ]:
from transformers import pipeline

pipe = pipeline("text-to-speech", model="suno/bark-small")
text = "[clears throat] This is a test ... and I just took a long pause."
output = pipe(text)

这里有一个代码片段，你可以在 notebook 中运行它来试听生成的音频：

In [ ]:
from IPython.display import Audio
Audio(output["audio"], rate=output["sampling_rate"])

想了解更多关于 Bark 和其他预训练 TTS 模型的示例，请参考我们的[音频课程](https://huggingface.co/learn/audio-course/chapter6/pre-trained_models)。

如果你想要微调一个 TTS 模型，目前 🤗 Transformers 中可用的文本转语音模型只有 [SpeechT5](https://huggingface.co/docs/transformers/main/en/tasks/model_doc/speecht5) 和 [FastSpeech2Conformer](https://huggingface.co/docs/transformers/main/en/tasks/model_doc/fastspeech2_conformer)，不过未来会添加更多模型。

SpeechT5 是在语音转文本和文本转语音数据的组合上进行预训练的，这使得它能够学习一个统一的隐含表示空间，这个空间由文本和语音共享。这意味着同一个预训练模型可以针对不同任务进行微调。此外，SpeechT5 通过 `x-vector` 说话者嵌入支持多个说话者。

本指南的其余部分将展示如何：

1. 将原本在英语语音上训练的 [SpeechT5](https://huggingface.co/docs/transformers/main/en/model_doc/speecht5) 微调到 [VoxPopuli](https://huggingface.co/datasets/facebook/voxpopuli) 数据集的荷兰语（nl）子集上。
2. 使用你微调后的模型进行推理，有两种方法：使用 pipeline 或直接使用。

在开始之前，请确保你已经安装了所有必要的库：

In [ ]:
pip install datasets soundfile speechbrain accelerate

需要从源代码安装 🤗Transformers，因为并非所有 SpeechT5 的功能都已合并到官方版本中：

In [ ]:
pip install git+https://github.com/huggingface/transformers.git

若要按照本指南操作，你还需要一个 GPU。如果你在 notebook 中工作，运行以下命令来检查是否有可用的 GPU：

In [ ]:
!nvidia-smi

对于 AMD GPU，可以使用：

In [ ]:
!rocm-smi

我们鼓励你登录你的 Hugging Face 账户，以便上传并与社区分享你的模型。当提示时，输入你的令牌进行登录：

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## 加载数据集

[VoxPopuli](https://huggingface.co/datasets/facebook/voxpopuli) 是一个大规模的多语种语音语料库，数据来源于2009-2020年欧洲议会活动录音。它包含15种欧洲语言的标记音频-转录数据。在本指南中，我们使用荷兰语子集，你也可以选择其他子集。

请注意，VoxPopuli 或任何其他自动语音识别（ASR）数据集可能不是训练 TTS 模型的最佳选择。那些对 ASR 有益的特性，比如过多的背景噪音，在 TTS 中通常是不希望出现的。所以，找到高质量、多语种、多说话者的 TTS 数据集确实相当有挑战性。

让我们加载数据：

In [ ]:
from datasets import load_dataset, Audio

dataset = load_dataset("facebook/voxpopuli", "nl", split="train")
len(dataset)

20968 个示例应该足够用于微调。SpeechT5 要求音频数据的采样率为 16 kHz，所以请确保数据集中的示例满足这一要求：

In [ ]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

## 预处理数据

首先，定义要使用的模型 checkpoint 并加载相应的处理器：

In [ ]:
from transformers import SpeechT5Processor

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)

### 为 SpeechT5 分词进行文本清理

首先，清理文本数据。你需要使用处理器的分词器部分来处理文本：

In [ ]:
tokenizer = processor.tokenizer

数据集示例包含 `raw_text` 和 `normalized_text` 特征。在决定使用哪个特征作为文本输入时，考虑到 `SpeechT5` 的分词器没有用于数字的任何标记。在 `normalized_text` 中，数字被写成文本形式。因此，它更适合作为输入文本，我们建议使用 `normalized_text`。

由于 `SpeechT5` 是在英语上训练的，它可能无法识别荷兰语数据集中的某些字符。如果保留原样，这些字符将被转换为 `<unk>` 标记。然而，在荷兰语中，某些字符如 `à` 用于强调音节。为了保留文本的意义，我们可以将这个字符替换为普通的 `a`。

为了识别不支持的标记，使用 `SpeechT5Tokenizer` 提取数据集中的所有唯一字符，该分词器以字符作为标记。为此，编写 `extract_all_chars` 映射函数，将所有示例的转录文本连接成一个字符串，并将其转换为字符集合。确保在 `dataset.map()` 中设置 `batched=True` 和 `batch_size=-1`，以便所有转录文本一次性提供给映射函数。

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["normalized_text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

现在你有两套字符集：一套是数据集中的词汇，另一套是分词器中的词汇。

为了识别数据集中任何不支持的字符，你可以取这两个集合的差集。结果集合将包含那些在数据集中但不在分词器中的字符，即不支持的字符集合。

In [ ]:
dataset_vocab - tokenizer_vocab

为了处理上一步中得到的不支持的字符，需要定义一个函数将这些字符映射到有效的标记。请注意，分词器中空格已经用 ▁ 替换，不需要单独处理。

In [ ]:
replacements = [
    ("à", "a"),
    ("ç", "c"),
    ("è", "e"),
    ("ë", "e"),
    ("í", "i"),
    ("ï", "i"),
    ("ö", "o"),
    ("ü", "u"),
]


def cleanup_text(inputs):
    for src, dst in replacements:
        inputs["normalized_text"] = inputs["normalized_text"].replace(src, dst)
    return inputs


dataset = dataset.map(cleanup_text)

现在你已经处理了文本中的特殊字符（不支持的字符），接下来该将注意力转向音频数据了。

### 说话者
VoxPopuli 数据集包含多位说话者的语音，但数据集中有多少位说话者呢？

为了确定这一点，我们可以统计具有独特特征的说话者的数量以及每位说话者为数据集贡献的示例数量。考虑到数据集中一共有 20,968 个示例，这些信息将帮助我们更好地理解说话者和示例在数据中的分布情况。

In [ ]:
from collections import defaultdict

speaker_counts = defaultdict(int)

for speaker_id in dataset["speaker_id"]:
    speaker_counts[speaker_id] += 1

通过绘制直方图，你可以了解每位说话者有多少数据。

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.hist(speaker_counts.values(), bins=20)
plt.ylabel("Speakers")
plt.xlabel("Examples")
plt.show()

直方图显示，数据集中大约三分之一的说话者拥有少于 100 个示例，而大约 10 位说话者拥有超过 500 个示例。

为了提高训练效率并平衡数据集，我们可以将数据限制在拥有 100 到 400 个示例的说话者范围内。

In [ ]:
def select_speaker(speaker_id):
    return 100 <= speaker_counts[speaker_id] <= 400


dataset = dataset.filter(select_speaker, input_columns=["speaker_id"])

让我们检查一下还剩下多少位说话者：

In [ ]:
len(set(dataset["speaker_id"]))

让我们看看还剩下多少个示例：

In [ ]:
len(dataset)

你现在大约有10,000 个示例，来自大约 40 位独特的说话者，这应该是足够的。

请注意，如果示例较长，那些只有少量示例的说话者实际上可能会拥有更多的音频数据。然而，确定每位说话者的总音频量需要遍历整个数据集，这是一个比较耗时的过程，涉及加载和解码每个音频文件。因此，我们选择在这里跳过这一步。

### 说话者嵌入
为了让 TTS 模型能够区分多位说话者，你需要为每个示例创建一个说话者嵌入。说话者嵌入是模型的额外输入，用于捕捉特定说话者的声音特征。

为了生成这些说话者嵌入，这里使用 `SpeechBrain` 的预训练模型 [spkrec-xvect-voxceleb](https://huggingface.co/speechbrain/spkrec-xvect-voxceleb)。

创建一个函数 `create_speaker_embedding()`，它接受一个输入音频波形，并输出一个包含相应说话者嵌入的 512 元素向量。

In [ ]:
import os
import torch
from speechbrain.inference.classifiers import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

需要注意的是，`speechbrain/spkrec-xvect-voxceleb` 模型是在 `VoxCeleb` 数据集的英语语音上训练的，而本指南中的训练示例是荷兰语。尽管我们相信这个模型仍能为我们的荷兰语数据集生成合理的说话者嵌入，但这种假设在所有情况下可能并不成立。

为了获得最佳结果，我们建议首先在目标语音上训练一个 `X-vector` 模型。这将确保模型能够更好地捕捉荷兰语中独特的声音特征。

### 处理数据集